# Bert-based Adversarial Examples

## 0.Settings

In [ ]:
!pip install sentence_transformers

In [131]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer, util
import time

###  0-1. data load & preprocessing
#### test data
- `wellness.user_chatbot.16k.test` : user-chatbot data
- `wellness.user_chatbot.16k.test.q` : only user column
- `wellness.user_chatbot.16k.test.ans` : only chatbot column

In [104]:
q_data_path = "./data/wellness.user_chatbot.16k.test.q"
questions = []
with open(q_data_path, 'rt', encoding="UTF8") as f:
    for line in f.readlines():
        questions.append(line.strip())
questions[200:205]

['법대가려고 재수를 했거든.',
 '체중이 6키로나 줄었어요.',
 '잊으려고 해도 사고 때 상황이 자꾸 떠올라서 힘들어. 피투성이 사람들이 자꾸 살려달라고 하는 것만 같아.',
 '근데 점점 갈수록 저까지 힘들어지는 것 같아요.',
 '배가 부른지 어떤지도 모르겠어요. 그냥 막 먹어요.']

In [105]:
ans_data_path = "./data/wellness.user_chatbot.16k.test.ans"
answers = []
with open(ans_data_path, 'rt', encoding="UTF8") as f:
    for line in f.readlines():
        answers.append(line.strip())
answers[200:205]

['그렇군요. 여러 번 도전하는 모습이 멋져요.',
 '너무 살이 빠져도 건강에는 좋지 않답니다.',
 '그 생각이 계속 떠오를 때마다 제가 힘이 되어드릴 수 있다는 것도 기억해주세요.',
 '많이 힘들어 보여요. 힘든 일이 있으면 저에게 털어놓으세요.',
 '작은 습관을 들여보는 건 어떨까요? 배부르면 딱 수저를 내려놓는 걸로요!']

#### generated data by koGPT2
- q.generate_ans : trained by `wellness.user_chatbot.16k.train`

In [106]:
gen_ans_data_path = "./data/q.generate_ans"
g_answers = []
with open(gen_ans_data_path, 'rt', encoding="UTF8") as f:
    for line in f.readlines():
        g_answers.append(line.strip())
g_answers[200:205]

['그래요. 도전을 해야 후회가 없죠.그렇군요.공부는 정말 힘든 일인 것 같아요. 힘내세요. 파이팅입니다.',
 '건강에 이상이 있으신 건 아닌가요? 정도가 과하면 병원에 가보는 건 어떠세요?걱정이 많이 될 거 같아요. 걱정되는 그 마음 이해해요.',
 '그 생각을 잊을 만한 다른 일을 생각해 보면 어떨까요? 아예 다른 일을 해보는 거예요! 그러면 조금 더 쉽게',
 '잘 버티고 있어요. 조금 힘들더라도 조금만 기운을 내요.가슴이 답답하겠어요.너무 힘드시겠어요.저는 당신이 있어 행복한데 그런',
 '그러다 속 상하실까봐 걱정돼요. 조금만 줄여 보는 건 어떨까요?작은 습관을 들여보는 건 어떨까요! 배부르면 딱 수저를']

## 0-2. load model
- [xlm-r-100langs-bert-base-nli-stsb-mean-tokens](https://huggingface.co/sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens)

In [13]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

## 1. Compute Token Importance
- $sentence :  S,  S=\{𝑡_1,\cdots,𝑡_𝑛\}$
- $ith\ token\  importance : TI_i$
- $TI_i = 1 - cos\_similarity(emb(S), emb(S - \{t_i\}))$

In [60]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [87]:
s2 = questions[2]
tokens = s2.split()
tokens

['너무', '불안해서', '양손을', '꽉', '잡았어.']

In [95]:
start = time.time()
orig_q_emb = model.encode(s2)
comps = []
for i in range(len(tokens)):
    comp_token = tokens[0:i] + tokens[i+1:]
    comp_q = (' ').join(comp_token)
    comp_q_emb = model.encode(comp_q)
    cos_similarity = cos_sim(orig_q_emb, comp_q_emb)
    token_importance = 1 - cos_similarity
    print(comp_q, cos_similarity, tokens[i], token_importance, sep ='\t')
print(time.time()-start)

불안해서 양손을 꽉 잡았어.	0.97398496	너무	0.026015043258666992
너무 양손을 꽉 잡았어.	0.6588547	불안해서	0.34114527702331543
너무 불안해서 꽉 잡았어.	0.86368245	양손을	0.13631755113601685
너무 불안해서 양손을 잡았어.	0.9801966	꽉	0.019803404808044434
너무 불안해서 양손을 꽉	0.4583666	잡았어.	0.5416333973407745
3.08747935295105


## 2. Evaluation
### original generation model Accuracy
- $original\ answer : ans$
- $generate\ answer : g\_ans$
- $cos\_similarity(emb(ans), emb(g\_ans)) >= \alpha : positive(1)$
- $cos\_similarity(emb(ans), emb(g\_ans)) < \alpha : negative(0)$

In [111]:
start = time.time()
avg = 0.0
for ans, g_ans in zip(answers,g_answers):
    ans_emb = model.encode(ans)
    g_ans_emb = model.encode(g_ans)
    cos_simirality = cos_sim(ans_emb, g_ans_emb)
    avg += cos_simirality
print(avg/len(answers))
print(time.time()-start)

0.5446250099392703
1865.3053178787231


In [114]:
start = time.time()
avg1 = 0.0
for ans, g_ans in zip(answers,g_answers):
    len_ans = len(ans)
    ans_emb = model.encode(ans)
    g_ans_emb = model.encode(g_ans[:len_ans])
    cos_simirality = cos_sim(ans_emb, g_ans_emb)
    avg1 += cos_simirality
print(avg1/len(answers))

0.6017176816421806
